# Pulsim Tutorial

This notebook demonstrates how to use Pulsim for circuit simulation, focusing on power electronics applications.

## Setup

First, make sure you have built the Python module:
```bash
make python
```

Then run this notebook from the `build/python` directory or add it to your Python path.

In [1]:
from pathlib import Path
import sys

_root = Path.cwd()
for _ in range(6):
    candidate = _root / 'build' / 'python'
    if candidate.is_dir():
        if str(candidate) not in sys.path:
            sys.path.insert(0, str(candidate))
        break
    _root = _root.parent

import numpy as np
import matplotlib.pyplot as plt
import pulsim as sl

print(f"Pulsim version: {sl.__version__}")

Pulsim version: 0.2.0


## 1. Simple RC Circuit

Let's start with a basic RC circuit to understand the API.

In [2]:
# Create a simple RC circuit
circuit = sl.Circuit()

# Add components: voltage source, resistor, capacitor
circuit.add_voltage_source("V1", "in", "0", 5.0)  # 5V DC
circuit.add_resistor("R1", "in", "out", 1000.0)   # 1k ohm
circuit.add_capacitor("C1", "out", "0", 1e-6, ic=0.0)  # 1uF, initial voltage = 0

# Validate the circuit
print(f"Nodes: {circuit.node_names()}")
print(f"Total variables: {circuit.system_size()}")

Nodes: ['in', 'out']
Total variables: 3


In [3]:
# Configure simulation
opts = sl.SimulationOptions()
opts.tstart = 0.0
opts.tstop = 5e-3    # 5ms (5 time constants for RC = 1ms)
opts.dt = 1e-6       # 1us initial timestep
opts.dtmax = 10e-6   # 10us max timestep
opts.use_ic = True   # Use initial conditions

# Run simulation
sim = sl.Simulator(circuit, opts)
result = sim.run_transient()

print(f"Simulation status: {result.final_status}")
print(f"Total steps: {len(result.time)}")
print(f"Available signals: {result.signal_names}")

Simulation status: SolverStatus.Success
Total steps: 5001
Available signals: ['V(in)', 'V(out)', 'I(V1)']


In [4]:
# Plot results
fig, ax = plt.subplots(figsize=(10, 5))

time_ms = np.array(result.time) * 1000  # Convert to ms

# Get signals from result data
data = result.to_dict()
v_in = data["signals"]["V(in)"]
v_out = data["signals"]["V(out)"]

ax.plot(time_ms, v_in, label="V(in)", linewidth=2)
ax.plot(time_ms, v_out, label="V(out)", linewidth=2)

# Add theoretical curve
tau = 1e-3  # RC = 1k * 1uF = 1ms
v_theory = 5.0 * (1 - np.exp(-np.array(result.time) / tau))
ax.plot(time_ms, v_theory, '--', label="Theoretical", alpha=0.7)

ax.set_xlabel("Time (ms)")
ax.set_ylabel("Voltage (V)")
ax.set_title("RC Circuit Step Response")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

/var/folders/0n/gs7hh4fj4bn8r3r4qsykw2840000gn/T/ipykernel_69367/2728103545.py:25: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 2. Buck Converter Simulation

Now let's simulate a more interesting circuit: a buck converter for power electronics.

In [5]:
# YAML netlist (for reference)
buck_netlist_yaml = """
schema: pulsim-v1
version: 1
components:
  - type: voltage_source
    name: Vdc
    nodes: [vcc, 0]
    waveform:
      type: dc
      value: 48.0
  - type: voltage_source
    name: Vpwm
    nodes: [ctrl, 0]
    waveform:
      type: pwm
      v_high: 10.0
      v_low: 0.0
      frequency: 20000
      duty: 0.5
      dead_time: 0.0
  - type: vcswitch
    name: S1
    nodes: [ctrl, vcc, sw]
    v_threshold: 5.0
    ron: 0.01
    roff: 1e9
  - type: diode
    name: D1
    nodes: [0, sw]
  - type: inductor
    name: L1
    nodes: [sw, out]
    value: 100u
  - type: capacitor
    name: C1
    nodes: [out, 0]
    value: 100u
  - type: resistor
    name: Rload
    nodes: [out, 0]
    value: 10.0
"""

# Build the circuit programmatically (equivalent to the YAML)
buck = sl.Circuit()
buck.add_voltage_source("Vdc", "vcc", "0", 48.0)

pwm = sl.PWMParams()
pwm.v_high = 10.0
pwm.v_low = 0.0
pwm.frequency = 20000
pwm.duty = 0.5
pwm.dead_time = 0.0
buck.add_pwm_voltage_source("Vpwm", "ctrl", "0", pwm)

buck.add_vcswitch("S1", "ctrl", "vcc", "sw", v_threshold=5.0, g_on=1.0/0.01, g_off=1.0/1e9)
buck.add_diode("D1", "0", "sw")
buck.add_inductor("L1", "sw", "out", 100e-6, ic=0.0)
buck.add_capacitor("C1", "out", "0", 100e-6, ic=0.0)
buck.add_resistor("Rload", "out", "0", 10.0)

print(f"Nodes: {buck.node_names()}")



Nodes: ['vcc', 'ctrl', 'sw', 'out']


In [6]:
# Simulate buck converter
opts = sl.SimulationOptions()
opts.tstart = 0.0
opts.tstop = 2e-3     # 2ms (40 switching cycles)
opts.dt = 100e-9      # 100ns initial timestep
opts.dtmax = 1e-6     # 1us max timestep
opts.use_ic = True
opts.abstol = 1e-9
opts.reltol = 1e-4

sim = sl.Simulator(buck, opts)
result = sim.run_transient()

print(f"Status: {result.final_status}")
print(f"Steps: {len(result.time)}")

Status: SolverStatus.Success
Steps: 20001


In [7]:
# Plot buck converter waveforms
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

time_us = np.array(result.time) * 1e6  # Convert to microseconds
data = result.to_dict()

# Plot 1: Control signal and switch node
ax1 = axes[0]
ax1.plot(time_us, data["signals"]["V(ctrl)"], label="V(ctrl)", alpha=0.8)
ax1.plot(time_us, data["signals"]["V(sw)"], label="V(sw)", alpha=0.8)
ax1.set_ylabel("Voltage (V)")
ax1.set_title("Buck Converter: Control and Switch Node")
ax1.legend(loc="upper right")
ax1.grid(True, alpha=0.3)

# Plot 2: Output voltage
ax2 = axes[1]
v_out = np.array(data["signals"]["V(out)"])
ax2.plot(time_us, v_out, label="V(out)", color="green", linewidth=1.5)
ax2.axhline(y=24.0, color='red', linestyle='--', alpha=0.5, label="Target (24V)")
ax2.set_ylabel("Voltage (V)")
ax2.set_title(f"Output Voltage (final: {v_out[-1]:.2f}V)")
ax2.legend(loc="lower right")
ax2.grid(True, alpha=0.3)

# Plot 3: Inductor current
ax3 = axes[2]
i_L = np.array(data["signals"]["I(L1)"])
ax3.plot(time_us, i_L, label="I(L1)", color="orange", linewidth=1.5)
ax3.set_xlabel("Time (μs)")
ax3.set_ylabel("Current (A)")
ax3.set_title(f"Inductor Current (final: {i_L[-1]:.2f}A)")
ax3.legend(loc="lower right")
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

/var/folders/0n/gs7hh4fj4bn8r3r4qsykw2840000gn/T/ipykernel_69367/3644926578.py:37: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [8]:
# Power loss summary (not available in the simplified Simulator wrapper)
print("Power loss summary not available in this tutorial build.")


Power loss summary not available in this tutorial build.


## 3. Zoom into Switching Waveforms

Let's look at a few switching cycles in detail.

In [9]:
# Zoom into last few cycles
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# Find indices for last 100us
t = np.array(result.time)
mask = t > (result.time[-1] - 100e-6)

time_zoom = t[mask] * 1e6
data = result.to_dict()

ax1 = axes[0]
v_sw = np.array(data["signals"]["V(sw)"])
ax1.plot(time_zoom, v_sw[mask], label="V(sw)", linewidth=1.5)
ax1.set_ylabel("Voltage (V)")
ax1.set_title("Switch Node Voltage (Zoomed)")
ax1.grid(True, alpha=0.3)
ax1.legend()

ax2 = axes[1]
i_L = np.array(data["signals"]["I(L1)"])
ax2.plot(time_zoom, i_L[mask], label="I(L1)", color="orange", linewidth=1.5)
ax2.set_xlabel("Time (μs)")
ax2.set_ylabel("Current (A)")
ax2.set_title("Inductor Current Ripple")
ax2.grid(True, alpha=0.3)
ax2.legend()

# Calculate ripple
i_ripple = i_L[mask].max() - i_L[mask].min()
print(f"Inductor current ripple: {i_ripple:.3f} A peak-to-peak")

plt.tight_layout()
plt.show()

Inductor current ripple: 6.248 A peak-to-peak


/var/folders/0n/gs7hh4fj4bn8r3r4qsykw2840000gn/T/ipykernel_69367/1379580218.py:33: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 4. Parsing Netlists from YAML

Pulsim uses a versioned YAML netlist format. You can keep YAML files alongside
your projects and build circuits programmatically for simulation.


In [10]:
# Example: YAML netlist (for reference)
netlist_yaml = """
schema: pulsim-v1
version: 1
components:
  - type: voltage_source
    name: V1
    nodes: [in, 0]
    waveform:
      type: pulse
      v_initial: 0
      v_pulse: 1
      t_delay: 0
      t_rise: 1e-9
      t_fall: 1e-9
      t_width: 1e-3
      period: 2e-3
  - type: resistor
    name: R1
    nodes: [in, n1]
    value: 10
  - type: inductor
    name: L1
    nodes: [n1, out]
    value: 100u
  - type: capacitor
    name: C1
    nodes: [out, 0]
    value: 10n
"""

# Build the circuit programmatically (equivalent to the YAML)
rlc = sl.Circuit()

pulse = sl.PulseParams()
pulse.v_initial = 0.0
pulse.v_pulse = 1.0
pulse.t_delay = 0.0
pulse.t_rise = 1e-9
pulse.t_fall = 1e-9
pulse.t_width = 1e-3
pulse.period = 2e-3

rlc.add_pulse_voltage_source("V1", "in", "0", pulse)
rlc.add_resistor("R1", "in", "n1", 10.0)
rlc.add_inductor("L1", "n1", "out", 100e-6)
rlc.add_capacitor("C1", "out", "0", 10e-9)

print(f"Parsed circuit: {rlc.num_nodes()} nodes")

# Simulate
opts = sl.SimulationOptions()
opts.tstop = 500e-6
opts.dt = 10e-9
opts.dtmax = 100e-9

sim = sl.Simulator(rlc, opts)
result = sim.run_transient()

# Plot
fig, ax = plt.subplots(figsize=(10, 5))
time_us = np.array(result.time) * 1e6
data = result.to_dict()
ax.plot(time_us, data["signals"]["V(out)"], label="V(out)")
ax.set_xlabel("Time (μs)")
ax.set_ylabel("Voltage (V)")
ax.set_title("RLC Circuit Response")
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

# Calculate resonant frequency
L = 100e-6
C = 10e-9
f0 = 1 / (2 * np.pi * np.sqrt(L * C))
print(f"Theoretical resonant frequency: {f0/1000:.1f} kHz")



Parsed circuit: 3 nodes


Theoretical resonant frequency: 159.2 kHz


/var/folders/0n/gs7hh4fj4bn8r3r4qsykw2840000gn/T/ipykernel_69367/3796067636.py:71: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 5. Thermal Simulation

Pulsim includes thermal modeling for power devices.

In [11]:
# Create thermal network for a MOSFET
thermal_network = sl.create_mosfet_thermal_model(
    0.5,   # Junction-to-case: 0.5 K/W
    0.3,   # Case-to-sink: 0.3 K/W
    2.0,   # Sink-to-ambient: 2.0 K/W
    "M1"
)

print(f"Total Rth (junction to ambient): {thermal_network.total_Rth():.2f} K/W")
print(f"Foster network stages: {thermal_network.num_stages()}")

# Create thermal simulator
thermal_sim = sl.ThermalSimulator(thermal_network, 25.0)
print(f"Initial junction temperature: {thermal_sim.Tj():.1f}°C")


Total Rth (junction to ambient): 2.80 K/W
Foster network stages: 3
Initial junction temperature: 25.0°C


In [12]:
# Simulate thermal response to power dissipation
power = 10.0  # 10W dissipation
dt = 1e-3     # 1ms timestep
t_total = 5.0 # 5 seconds

times = []
temps = []

t = 0.0
while t < t_total:
    thermal_sim.step(power, dt)
    times.append(t)
    temps.append(thermal_sim.Tj())
    t += dt

# Plot thermal response
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(times, temps, linewidth=2)

# Theoretical steady-state
t_ss = 25.0 + power * thermal_network.total_Rth()
ax.axhline(y=t_ss, color='red', linestyle='--', alpha=0.7, 
           label=f"Steady-state: {t_ss:.1f}°C")

ax.set_xlabel("Time (s)")
ax.set_ylabel("Junction Temperature (°C)")
ax.set_title(f"MOSFET Thermal Response (P = {power}W)")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Final junction temperature: {temps[-1]:.1f}°C")


Final junction temperature: 51.4°C


/var/folders/0n/gs7hh4fj4bn8r3r4qsykw2840000gn/T/ipykernel_69367/3957674860.py:31: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 6. Exporting Results

Simulation results can be easily exported for further analysis.

In [13]:
# Convert results to dictionary (useful for pandas/CSV export)
data = result.to_dict()

print("Result dictionary keys:", list(data.keys()))
print(f"Number of time points: {len(data['time'])}")
print(f"Signals available: {list(data['signals'].keys())}")

Result dictionary keys: ['time', 'signals']
Number of time points: 50001
Signals available: ['V(in)', 'V(n1)', 'V(out)', 'I(V1)', 'I(L1)']


In [14]:
# Example: Create a pandas DataFrame
try:
    import pandas as pd
    
    df = pd.DataFrame({
        'time': data['time'],
        **data['signals']
    })
    
    print(df.head())
    print(f"\nDataFrame shape: {df.shape}")
    
    # Could save to CSV:
    # df.to_csv('simulation_results.csv', index=False)
except ImportError:
    print("pandas not installed - skipping DataFrame example")

           time  V(in)     V(n1)    V(out)     I(V1)     I(L1)
0  0.000000e+00    0.0  0.000000  0.000000  0.000000  0.000000
1  1.000000e-08    1.0  0.999500  0.000025 -0.000050  0.000050
2  2.000000e-08    1.0  0.998501  0.000125 -0.000150  0.000150
3  3.000000e-08    1.0  0.997504  0.000325 -0.000250  0.000250
4  4.000000e-08    1.0  0.996507  0.000624 -0.000349  0.000349

DataFrame shape: (50001, 6)


## Summary

This tutorial covered:

1. **Basic circuit construction** - Creating circuits programmatically with resistors, capacitors, inductors, and voltage sources
2. **Transient simulation** - Running time-domain simulations with configurable options
3. **Power electronics** - Buck converter simulation with switches, diodes, and PWM control
4. **YAML netlist format** - Versioned YAML netlists for interchange and documentation
